Install the required libraries.

*   The geopandas library allows us to work with geospatial data [(GeoPandas, 2013)](https://geopandas.org/en/stable/)
*   The pandas library allows us to work with data in a dataframe [(pandas - Python Data Analysis Library, 2021)](https://pandas.pydata.org/)
*   The plotly library allows us to work with interactive data visualisations [(Plotly, 2022)](https://plotly.com/python/)
*   The json library allows us to work with JSON data [(Python, 2022)](https://docs.python.org/3/library/json.html)
*   The pyproj library allows us to work with cartographic projects [(PyPi, 2022)](https://pypi.org/project/pyproj/)




In [ ]:
import geopandas as gpd
import pandas as pd
import plotly.express as px
import json
import pyproj

# Interactive choropleth map of precincts

Code is adapted from a blog on Medium [(Sanghvi, 2020)](https://medium.com/tech-carnot/interactive-map-based-visualization-using-plotly-44e8ad419b97).

First we need to convert the shapefile to a geojson file as plotly doesn't read shapefiles like pandas.

In [ ]:
# Read the file stored
map_df = gpd.read_file('data/georgia_shapefiles_2020_precinct/cb_2020_13_vtd_500k.shp')

# Export it as GeoJSON
map_df.to_file("data/json/georgia_geojson_2020_precinct.json", driver='GeoJSON')

Viewing the json file.

In [ ]:
with open('data/json/georgia_geojson_2020_precinct.json') as f:
  precinct = json.load(f)

In [ ]:
precinct["features"][0]['properties']

Creating the visualisation by reading the shapefile and joining the race data with the shapefile.

In [ ]:
# Read the shapefile
filepath = 'data/json/georgia_geojson_2020_precinct.json'
map_df = gpd.read_file(filepath)
map_df.to_crs(pyproj.CRS.from_epsg(4326), inplace=True)

df = pd.read_csv("data/race_precinct_data/cleaned_georgia_race_precinct_densities.csv")

# Join the geodataframe with the cleaned up csv dataframe
merged = map_df.set_index('AFFGEOID20').join(df.set_index('id'))
merged.set_index('Area Name', inplace=True)
merged.head()



In [ ]:
merged.isnull().sum()

To test it out and show just one race with code adapted from [(Plotly, 2019a](https://plotly.com/python/choropleth-maps/); [Plotly, 2019b](https://plotly.com/python/mapbox-county-choropleth/)).

In [ ]:
fig = px.choropleth_mapbox(merged, geojson=merged.geometry, locations=merged.index, mapbox_style="open-street-map", zoom = 5, center = {"lat": 32.6047297, "lon": -83.3045263}, color="Population Density: White", labels={'AFFGEOID20':'Precinct'})
fig.update_geos(fitbounds="locations", visible=False)

Making the visualisation with dropdown menus to allow you to select different races with code adapted from the plotly community forum [(Plotly Community, 2021)](https://community.plotly.com/t/creating-a-dropdown-slider-for-a-choropleth-map-with-plotly-express/49370).

In [ ]:
df = merged

fig = px.choropleth_mapbox(df, geojson=merged.geometry,
                    locations=merged.index, zoom = 5, center = {"lat": 32.6047297, "lon": -83.3045263}, color="Population Density: White", labels={'Population Density: White':'Population Density'}, 
                    mapbox_style="open-street-map", color_continuous_scale="deep"
                   )
fig.update_geos(fitbounds="locations", visible=False)


button1 =  dict(method = "restyle",
                args = [{'z': [ df["Population Density: White"] ]}],
                label = "White")
button2 =  dict(method = "restyle",
                args = [{'z': [ df["Population Density: Black"] ] }],
                label = "Black")
button3 =  dict(method = "restyle",
                args = [{'z': [ df["Population Density: Hispanic"] ] }],
                label = "Hispanic")
button4 =  dict(method = "restyle",
                args = [{'z': [ df["Population Density: Asian"] ] }],
                label = "Asian")  
button5 =  dict(method = "restyle",
                args = [{'z': [ df["Population Density: Others"] ] }],
                label = "Others")
button6 =  dict(method = "restyle",
                args = [{'z': [ df["Population Density: Mixed"] ] }],
                label = "Mixed")                              

fig.update_layout(width=700,
                  coloraxis_colorbar_thickness=23,
                  updatemenus=[dict(y=0.9,
                                    x=0.275,
                                    xanchor='right',
                                    yanchor='top',
                                    active=0,
                                    buttons=[button1, button2, button3, button4, button5, button6])
                              ]) 



In [ ]:
fig.write_html('html_files/plotly_choropleth_map_race_densities_precinct.html', include_plotlyjs="cdn", full_html = False)

# Interactive choropleth map of counties

Similar steps were conducted to create the interactive map for counties.

In [ ]:
# Read the shapefilefile
map_df_county = gpd.read_file('data/georgia_shapefiles_2020_county/georgia_county.shp')

# Export the shapefile as GeoJSON
map_df_county.to_file("data/json/georgia_geojson_2020_county.json", driver='GeoJSON')

# Read the GeoJSON
filepath_county = 'data/json/georgia_geojson_2020_county.json'
map_df_county = gpd.read_file(filepath_county)
map_df_county.to_crs(pyproj.CRS.from_epsg(4326), inplace=True)
map_df_county.columns = map_df_county.columns.map(str)



Unlike the shapefile for precincts, the geo id's in the shapefile for counties consist of only 5 characters, which are the last 5 characters in a county's full geo id. Thus, these last 5 characters must be extracted from the race data.

In [ ]:
map_df_county.head()

In [ ]:
# Read the racial densities csv
df_county = pd.read_csv("data/race_county_data/cleaned_georgia_race_county.csv")

# Extract the last five characters of the id
df_county['id']=df_county['id'].str.slice(start=-5)

In [ ]:
# Merge the geodataframe with the cleaned up csv dataframe
merged_county = map_df_county.set_index('GEOID').join(df_county.set_index('id'))
merged_county.set_index('Area Name', inplace=True)

merged_county.head()

In [ ]:
print(merged_county.shape)

In [ ]:
df_county = merged_county

fig = px.choropleth_mapbox(df_county, geojson=merged_county.geometry,
                    locations=merged_county.index, zoom = 5, center = {"lat": 32.6047297, "lon": -83.3045263}, color="Population Density: White", labels={'Population Density: White':'Population Density'}, 
                    mapbox_style="open-street-map", color_continuous_scale="deep"
                   )
fig.update_geos(fitbounds="locations", visible=False)


button1 =  dict(method = "restyle",
                args = [{'z': [ df_county["Population Density: White"] ]}],
                label = "White")
button2 =  dict(method = "restyle",
                args = [{'z': [ df_county["Population Density: Black"] ] }],
                label = "Black")
button3 =  dict(method = "restyle",
                args = [{'z': [ df_county["Population Density: Hispanic"] ] }],
                label = "Hispanic")
button4 =  dict(method = "restyle",
                args = [{'z': [ df_county["Population Density: Asian"] ] }],
                label = "Asian")  
button5 =  dict(method = "restyle",
                args = [{'z': [ df_county["Population Density: Others"] ] }],
                label = "Others")
button6 =  dict(method = "restyle",
                args = [{'z': [ df_county["Population Density: Mixed"] ] }],
                label = "Mixed")                              

fig.update_layout(width=700,
                  coloraxis_colorbar_thickness=23,
                  updatemenus=[dict(y=0.9,
                                    x=0.275,
                                    xanchor='right',
                                    yanchor='top',
                                    active=0,
                                    buttons=[button1, button2, button3, button4, button5, button6])
                              ]) 


In [ ]:
fig.write_html('html_files/plotly_choropleth_map_race_densities_county.html', include_plotlyjs="cdn", full_html = False)

Citations:

Plotly. 2022. Plotly. [online] Available at: <https://plotly.com/python/> [Accessed January 11, 2022]. 

Plotly. (2019a). Chloropleth maps in Python. [online] Available at: <https://plotly.com/python/choropleth-maps/> (Accessed December 24, 2021).

Plotly. (2019b). Mapbox Choropleth Maps in Python. [online] Available at: <https://plotly.com/python/mapbox-county-choropleth/> (Accessed December 24, 2021).

Plotly Community, 2021. Creating a dropdown + slider for a choropleth map with plotly.express? Plotly Community Forum. [online] Available at: <https://community.plotly.com/t/creating-a-dropdown-slider-for-a-choropleth-map-with-plotly-express/49370> [Accessed December 26, 2021]. 

PyPi, 2022. Pyproj. PyPI. [online] Available at: <https://pypi.org/project/pyproj/> [Accessed January 11, 2022]. 

Python, 2022. JSON - JSON encoder and decoder. Python 3.10.1 documentation. [online] Available at: <https://docs.python.org/3/library/json.html> [Accessed January 11, 2022]. 

Sanghvi, Y., 2020. Interactive Map-based Visualization using Plotly. Medium. [online] Available at: <https://medium.com/tech-carnot/interactive-map-based-visualization-using-plotly-44e8ad419b97> [Accessed December 24, 2021]. 